# Matching Compustat firm with assignee

### Compustat
- 2010년 상장사 재무정보
- nameprocessing 완료 (지난 프로젝트에서)

### matching 방법
- 이름 기준으로 matching (key가 딱히 없음)
- nameprocessing 결과로 1차 matching
- matching 실패한 assignee 중 num_patent 기준 상위 n%에 대해 manual matching
- match 후 key 부여 (index = key로 두면 될 듯)

### 1. assignee nameprocessing

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

def nameprocessing(s):
    s = str(s)
    
    s = s.replace(".","")
    s = s.replace(",","")    
    s = s.replace(";","")
    s = ''.join(s.split()).lower()
    
    s = s.replace("incorporated","inc")
    s = s.replace("limited","ltd")
    s = s.replace("gmbh","")
    
    rm=['inc','ltd','llc','pte','plc','kft']
    for i in range(0,len(s)):
        if s[-3:] in rm:
            s=s[:-3]
        else:
            break

    return s

In [2]:
df = pd.read_csv('assignee_profile.csv')
df['new_assignee'] = df['assignee'].apply(nameprocessing)
df.to_csv('assignee_new.csv', index=False)
df.iloc[0:5,:]

,assignee,num_patent,new_assignee
0,INTERNATIONAL BUSINESS MACHINES CORPORATION,5614,internationalbusinessmachinescorporation
1,SAMSUNG ELECTRONICS CO LTD,4128,samsungelectronicsco
2,MICROSOFT CORPORATION,3011,microsoftcorporation
3,PANASONIC CORPORATION,2128,panasoniccorporation
4,CANON KABUSHIKI KAISHA,2044,canonkabushikikaisha


### 2. matching

1. assignee를 new_assingee 기준으로 뭉치기
2. compustat과 new_assignee 기준으로 match

In [3]:
a=len(df)
df = df.groupby('new_assignee',as_index=False).sum()
df.sort_values('num_patent', ascending=False, inplace=True)

print a, len(df)

7753 7664


약 100개 정도 더 뭉치기 성공

In [4]:
com = pd.read_csv('compustat_10.csv')[['new_assignee','conm']]
df = df.merge(com, on='new_assignee', how='left', sort=False)
df.index.names = ['PCkey']
df.drop_duplicates(inplace=True)   #I don't know why but there are some duplicates
df.to_csv('patent_compustat_key.csv')

In [5]:
print df['num_patent'].median(), df['num_patent'].mean()

2.0 14.9595722483


### 3. manual processing
- 일단 num_patent가 평균 보다 큰 기업의 경우(14개) matching 시도 (엑셀로)
- median이 2.0이니 흠...